In [0]:
# ---------------------------------------------------------
# CELL 1: SETUP & LOAD SILVER DATA
# ---------------------------------------------------------
storage_name = "stfoodsafetysk" # <--- CHECK THIS
storage_key = "KEY_REMOVED_FOR_SECURITY" 

spark.conf.set(
    f"fs.azure.account.key.{storage_name}.dfs.core.windows.net",
    storage_key
)

# Read the Unified Silver Data
df_silver = spark.read.format("parquet") \
    .load(f"abfss://datalake@{storage_name}.dfs.core.windows.net/silver/unified_inspections")

print("✅ Silver Data Loaded. Count:", df_silver.count())

✅ Silver Data Loaded. Count: 11000


In [0]:
# ---------------------------------------------------------
# CELL 2: DEFINE THE RISK CLASSIFIER (LOCAL NLP)
# ---------------------------------------------------------
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

def classify_risk_logic(text):
    if not text:
        return "Unknown"
    
    text = text.lower()
    
    # 1. High Risk: Pests
    if any(x in text for x in ['mice', 'rat', 'roach', 'insect', 'pest', 'droppings', 'fly', 'vermin']):
        return "Pest Infestation"
    
    # 2. High Risk: Temperature
    elif any(x in text for x in ['temp', 'cool', 'refriger', 'hot hold', 'thermometer', 'thaw']):
        return "Temperature Violation"
    
    # 3. Medium Risk: Hygiene
    elif any(x in text for x in ['hand', 'soap', 'sink', 'wash', 'sanitiz', 'glove', 'hair']):
        return "Hygiene & Sanitation"
    
    # 4. Medium Risk: Infrastructure
    elif any(x in text for x in ['plumb', 'leak', 'water', 'pipe', 'sewage', 'floor', 'wall', 'ceiling']):
        return "Plumbing/Infrastructure"
    
    # 5. Low Risk: Paperwork
    elif any(x in text for x in ['certif', 'license', 'permit', 'manager', 'sign', 'post']):
        return "Paperwork/Compliance"
        
    else:
        return "General Maintenance"

# Turn Python function into a Spark Function (UDF)
risk_udf = udf(classify_risk_logic, StringType())

print("✅ Classification Logic Registered.")

✅ Classification Logic Registered.


In [0]:
# ---------------------------------------------------------
# CELL 3: APPLY LOGIC & SAVE TO GOLD
# ---------------------------------------------------------
# 1. Apply the logic to create "risk_category"
df_gold = df_silver.withColumn("risk_category", risk_udf(col("violations")))

# 2. Select columns for Power BI
df_final = df_gold.select(
    "restaurant_name",
    "city",
    "inspection_date",
    "result",
    "risk_category",  # <--- The New Column
    "latitude",
    "longitude",
    "violations"
)

# 3. Save to GOLD Layer (Parquet)
gold_path = f"abfss://datalake@{storage_name}.dfs.core.windows.net/gold/risk_analytics"

df_final.write \
    .format("parquet") \
    .mode("overwrite") \
    .save(gold_path)

print(f"SUCCESS! Gold Data Saved to: {gold_path}")
display(df_final.limit(10))

SUCCESS! Gold Data Saved to: abfss://datalake@stfoodsafetysk.dfs.core.windows.net/gold/risk_analytics


restaurant_name city inspection_date result risk_category latitude longitude violations TIENDA Y RESTAURANTE LA CAMPIñA SALvadorena Dallas 2016-10-03 Pass Pest Infestation 32.895847 -96.881391 228.34 Management and Personnel. Duties. The person in charge shall ensure that: (8) employees are using proper methods to rapidly cool TCS foods that are not held hot or are not for consumption within four hours, through daily oversight of the employees' routine monitoring of food temperatures during cooling; | 228.75 Food. Time and temperature control. (e) Cooling methods. (1) Cooling shall be accomplished in accordance with the time and temperature criteria specified in subsection (d) of this section by using one or more of the following methods based on the type of food being cooled: (E) using containers that facilitate heat transfer; | 228.75 Food. Time and temperature control. (f) Time/temperature controlled for safety food, hot and cold holding. (1) Except during preparation, cooking, or cooling, or when time is used as the public health control as specified in subsection (i) of this section, and except as specified in paragraphs (2) and (3) of this subsection, TCS food shall be maintained: (B) at 5 degrees Celsius (41 degrees Fahrenheit) or less; | 228.42 Management and Personnel. Food Contamination Prevention (a) Eating, drinking, or using tobacco. (1) except as specified in paragraph (2) of this subsection, an employee shall eat, drink, or use any form of tobacco only in designated areas where the contamination of exposed food; clean equipment, utensils, and linens; unwrapped single-service and single-use articles; or other items needing protection cannot result. | 228.69 Food. Preventing contamination from the premises. (a) Food Storage. (1) Except as specified in paragraphs (2) and (3) of this subsection, food shall be protected from contamination by storing the food: (C) at least 15 centimeters (6 inches) above the floor. | 228.186 Physical Facilities. Premises, buildings, systems, rooms, fixtures, equipment, devices, and materials. (k) Controlling pests. The presence of insects, rodents, and other pests shall be controlled to eliminate their presence on the premises by: (3) using methods, if pests are found, such as trapping devices or other means of pest control as specified in õ228.204(b) and õ228.208(b) and (c) of this title; and | | | | | 228.75 Food. Time and temperature control. (d) Cooling. (1) Cooked temperature/time controlled for safety food shall be cooled: (A) within two hours, from 57 degrees Celsius (135 degrees Fahrenheit) to 21 degrees C (70 degrees Fahrenheit); and | | | | | | | 228.111 Equipment, Utensils, and Linens. Equipment, maintenance and operation. (n) Manual and mechanical warewashing equipment, chemical sanitization - temperature, pH, concentration, and hardness. A chemical sanitizer used in a sanitizing solution for a manual or mechanical operation at contact times specified in õ228.118(3) of this title shall meet the criteria in õ228.206(a) of this title, shall be used in accordance with the EPA-approved manufacturer's label use instructions, and shall be used as follows: (1) a chlorine solution shall have a minimum temperature based on the concentration and pH of the solution as listed in the following chart: Figure: 25 TAC õ228.111(n)(1) | 228.149 Water, Plumbing, and Waste. Plumbing, operation and maintenance. (a) Using a handwashing facility. (1) A handwashing facility shall be maintained so that it is accessible at all times for employee use. | Sec. 17-2.2(c)(1)(D) (c) Registered food service managers. (1) Registered food service managers required. (D) A food establishment shall have one registered food service manager employed and present in the establishment during all hours of operation, except that a registered food service manager serving multiple food establishments as authorized by Section 17-2.2(c)(1)(C) must only be present in the building in which the food establishment is located

In [0]:
# Save as CSV (single file)
df_final.coalesce(1).write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(f"abfss://datalake@{storage_name}.dfs.core.windows.net/gold/risk_analytics_csv")